## 1. Importación de Librerías

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import sys

# Añadir directorio raíz al path
root_dir = Path.cwd().parent
sys.path.append(str(root_dir))

from src.config import Config
from src.data import get_data_loaders, get_class_distribution
from src.models import create_model
from src.training import Trainer, EarlyStopping
from src.evaluation import evaluate_model, plot_confusion_matrix, plot_training_history
from src.utils import plot_sample_images

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Configuración del Proyecto

In [ ]:
# Crear directorios necesarios
Config.create_dirs()

# Mostrar configuración
print("Configuración del Proyecto:")
print(f"Clases seleccionadas: {Config.SELECTED_CLASSES}")
print(f"Número de clases: {Config.NUM_CLASSES}")
print(f"Modelo: {Config.MODEL_NAME}")
print(f"Tamaño de entrada: {Config.INPUT_SIZE}")
print(f"Batch size: {Config.BATCH_SIZE}")
print(f"Epochs: {Config.NUM_EPOCHS}")
print(f"Learning rate: {Config.LEARNING_RATE}")
print(f"Device: {Config.DEVICE}")

## 3. Carga y Exploración de Datos

In [ ]:
# Cargar datasets
print("Cargando datasets...")
train_loader, val_loader, test_loader = get_data_loaders()

# Distribución de clases
train_dist = get_class_distribution(train_loader)
print(f"\nDistribución de clases en entrenamiento: {train_dist}")

In [ ]:
# Visualizar muestras del dataset
print("Visualizando muestras del dataset...")
plot_sample_images(train_loader, num_samples=12)

## 4. Construcción del Modelo - Versión 1 (Simple)

Clasificador simple sin Batch Normalization ni Dropout.

In [ ]:
# Crear modelo Versión 1
print("\nCreando Modelo Versión 1 - Clasificador Simple")
model_v1 = create_model(variant='simple', pretrained=True)

# Verificar arquitectura
print("\nÚltimas capas del modelo:")
print(model_v1.mobilenet.classifier)

## 5. Entrenamiento del Modelo - Versión 1

In [ ]:
# Configurar dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Entrenando en: {device}")

# Configurar entrenamiento
criterion = nn.CrossEntropyLoss()
optimizer_v1 = optim.Adam(model_v1.parameters(), lr=Config.LEARNING_RATE, weight_decay=Config.WEIGHT_DECAY)
scheduler_v1 = optim.lr_scheduler.StepLR(optimizer_v1, step_size=10, gamma=0.1)

# Early Stopping
early_stopping_v1 = EarlyStopping(
    patience=Config.PATIENCE,
    min_delta=Config.MIN_DELTA,
    mode='min'
)

# Crear trainer
trainer_v1 = Trainer(
    model=model_v1,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_v1,
    device=device,
    scheduler=scheduler_v1,
    early_stopping=early_stopping_v1,
    save_dir=Config.MODELS_DIR
)

In [ ]:
# Entrenar modelo V1
history_v1 = trainer_v1.train(num_epochs=Config.NUM_EPOCHS, model_name='mobilenet_v3_simple')

In [ ]:
# Visualizar curvas de aprendizaje
plot_training_history(history_v1)

## 6. Evaluación del Modelo - Versión 1

In [ ]:
# Evaluar en conjunto de prueba
print("Evaluando Modelo Versión 1 en conjunto de prueba...")
results_v1 = evaluate_model(model_v1, test_loader, device=device)

In [ ]:
# Matriz de confusión
plot_confusion_matrix(results_v1['confusion_matrix'], normalize=True)

## 7. Construcción del Modelo - Versión 2 (Extendido)

Clasificador extendido con capas ocultas, Batch Normalization y Dropout.

In [ ]:
# Crear modelo Versión 2
print("\nCreando Modelo Versión 2 - Clasificador Extendido")
model_v2 = create_model(variant='extended', pretrained=True)

# Verificar arquitectura
print("\nArquitectura del clasificador:")
print(model_v2.mobilenet.classifier)

## 8. Entrenamiento del Modelo - Versión 2

In [ ]:
# Configurar entrenamiento V2
optimizer_v2 = optim.Adam(model_v2.parameters(), lr=Config.LEARNING_RATE, weight_decay=Config.WEIGHT_DECAY)
scheduler_v2 = optim.lr_scheduler.StepLR(optimizer_v2, step_size=10, gamma=0.1)

early_stopping_v2 = EarlyStopping(
    patience=Config.PATIENCE,
    min_delta=Config.MIN_DELTA,
    mode='min'
)

trainer_v2 = Trainer(
    model=model_v2,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_v2,
    device=device,
    scheduler=scheduler_v2,
    early_stopping=early_stopping_v2,
    save_dir=Config.MODELS_DIR
)

In [ ]:
# Entrenar modelo V2
history_v2 = trainer_v2.train(num_epochs=Config.NUM_EPOCHS, model_name='mobilenet_v3_extended')

In [ ]:
# Visualizar curvas de aprendizaje
plot_training_history(history_v2)

## 9. Evaluación del Modelo - Versión 2

In [ ]:
# Evaluar en conjunto de prueba
print("Evaluando Modelo Versión 2 en conjunto de prueba...")
results_v2 = evaluate_model(model_v2, test_loader, device=device)

In [ ]:
# Matriz de confusión
plot_confusion_matrix(results_v2['confusion_matrix'], normalize=True)

## 10. Comparación de Variantes

In [ ]:
import pandas as pd

# Crear tabla comparativa
comparison = pd.DataFrame({
    'Métrica': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'Versión 1 (Simple)': [
        f"{results_v1['accuracy']:.4f}",
        f"{results_v1['precision_avg']:.4f}",
        f"{results_v1['recall_avg']:.4f}",
        f"{results_v1['f1_avg']:.4f}"
    ],
    'Versión 2 (Extendido)': [
        f"{results_v2['accuracy']:.4f}",
        f"{results_v2['precision_avg']:.4f}",
        f"{results_v2['recall_avg']:.4f}",
        f"{results_v2['f1_avg']:.4f}"
    ]
})

print("\nComparación de Variantes:")
print(comparison.to_string(index=False))

## 11. Conclusiones y Análisis Final

### Análisis de Resultados

En esta sección se deben documentar los hallazgos del experimento:

#### Comparación de Desempeño entre Variantes

**Versión 1 (Simple):**
- Ventajas: Menor número de parámetros, entrenamiento más rápido, menos propenso a overfitting en datasets pequeños
- Desventajas: Capacidad de representación limitada

**Versión 2 (Extendido):**
- Ventajas: Mayor capacidad de representación, Batch Normalization estabiliza entrenamiento, Dropout reduce overfitting
- Desventajas: Mayor tiempo de entrenamiento, más parámetros, requiere más datos

#### Análisis del Impacto de Batch Normalization y Dropout

**Batch Normalization:**
- Normaliza las activaciones entre capas
- Permite usar learning rates más altos
- Reduce la dependencia de la inicialización de pesos
- Actúa como regularizador suave

**Dropout:**
- Previene co-adaptación de neuronas
- Reduce overfitting
- Mejora la generalización del modelo
- Probabilidades incrementales (0.2 → 0.5) aumentan regularización progresivamente

#### Estabilidad del Entrenamiento

- La versión con BN tiende a converger más rápido
- Las curvas de pérdida son más suaves con regularización
- Early stopping evita sobreentrenamiento

#### Diferencias de Desempeño entre Variantes

Comparar las métricas obtenidas:
- ¿Qué modelo tiene mejor accuracy?
- ¿Qué clases son más difíciles de clasificar?
- ¿Hay overfitting en alguna variante?

#### Limitaciones Observadas

**Trabajando con recursos limitados (Google Colab):**
- Tiempo de ejecución limitado
- Memoria GPU restringida
- Necesidad de reducir batch size o número de epochs
- Interrupciones en sesiones largas

**Limitaciones del dataset:**
- Desbalance de clases (si aplica)
- Variabilidad en calidad de imágenes
- Fondos y contextos diversos

**Limitaciones del modelo:**
- MobileNetV3 está optimizado para eficiencia, no máxima precisión
- Transfer Learning depende de la similitud con ImageNet

#### Recomendaciones para Mejoras Futuras

1. **Data Augmentation más agresivo:** Rotaciones, recortes, cambios de color
2. **Fine-tuning:** Descongelar capas del backbone progresivamente
3. **Ensemble de modelos:** Combinar predicciones de ambas variantes
4. **Optimización de hiperparámetros:** Grid search o búsqueda bayesiana
5. **Aumento del dataset:** Más imágenes de entrenamiento
6. **Probar otras arquitecturas:** EfficientNet, ResNet, Vision Transformer

## 12. Análisis de Errores de Predicción

Analicemos los errores que comete cada modelo para entender mejor su comportamiento.

In [ ]:
# Analizar errores de V1
from src.utils import analyze_predictions_errors, plot_prediction_errors

print("Analizando errores de Versión 1...")
errors_v1 = analyze_predictions_errors(model_v1, test_loader, device=device, num_samples=9)

print(f"\nSe encontraron {len(errors_v1)} errores para mostrar")
plot_prediction_errors(errors_v1)

In [ ]:
# Analizar errores de V2
print("Analizando errores de Versión 2...")
errors_v2 = analyze_predictions_errors(model_v2, test_loader, device=device, num_samples=9)

print(f"\nSe encontraron {len(errors_v2)} errores para mostrar")
plot_prediction_errors(errors_v2)

## 13. Gráfico Comparativo de Desempeño

In [ ]:
from src.utils import create_performance_comparison_plot

# Crear gráfico comparativo
create_performance_comparison_plot(results_v1, results_v2)

## 14. Guardar Resultados del Experimento

In [ ]:
from src.utils import ResultsSaver, save_experiment_summary

# Crear saver
saver = ResultsSaver(base_dir=Path('results'))

# Guardar resultados de ambas variantes
print("Guardando resultados de Versión 1...")
save_experiment_summary(results_v1, history_v1, "transfer_learning_frutas", "simple")

print("\nGuardando resultados de Versión 2...")
save_experiment_summary(results_v2, history_v2, "transfer_learning_frutas", "extended")

# Guardar reporte comparativo
print("\nCreando reporte comparativo...")
saver.save_comparison_report(results_v1, results_v2, "transfer_learning_frutas")

print("\n✓ Todos los resultados han sido guardados exitosamente!")

## 15. Resumen de Modelos

In [ ]:
from src.utils import print_model_summary

# Resumen de Versión 1
print_model_summary(model_v1, "Modelo Versión 1 (Simple)")

# Resumen de Versión 2
print_model_summary(model_v2, "Modelo Versión 2 (Extendido)")